# SPOTlight pipeline

The link to the tutorial from which this notebook is adapted can be found [here](https://marcelosua.github.io/SPOTlight/articles/SPOTlight_kidney.html#workflow). Certain changes have been made to adapt the different object types that this notebook is dealing with.

## Load the required libraries

`reticulate` must be installed on the conda environment that contains the R kernel.
The next step sets up `reticulate` to use the 'right' python. Following this, `reticulate` 

In [ ]:
library(SPOTlight)
library(rhdf5)
Sys.setenv(RETICULATE_PYTHON = "/media/gambino/students_workdir/ibp/gautam/miniconda3/envs/r_kernel/bin/python")
library(reticulate)
use_condaenv(condaenv = "r_kernel", conda  = "/media/gambino/students_workdir/ibp/gautam/miniconda3/bin/conda")
library(anndata)
library(SpatialExperiment)
library(SingleCellExperiment)
library(scater)
library(scran)
library(zellkonverter)
library(Seurat)
library(dplyr)
library(Matrix)
library(ggcorrplot)
library(Polychrome)

spatial_data_path <- "path/to/spatial_data/sample/outs"
spatial_sample_name <- "sample"
scRNAseq_data_path <- "path/to/scRNAseq_data"

## Read in the Visium file

In [2]:
spe <- read10xVisium(
  samples = spatial_data_path,
  sample_id = sample,
  type = "HDF5",
  data = "filtered",
  images = "hires",
  load = FALSE
)

## Change the row names of the Visium count matrix to refer to the genes' names rather than their Ensemble IDs

In [3]:
rownames(spe) <- rowData(spe)$symbol

## Read the scRNA-seq data

In [4]:
adata <- read_h5ad(scRNAseq_data_path)

## Subset the data appropriately

SPOTlight was made to work very easily with the Bioconductor RNA-seq ecosystem (`SpatialExperiment`, `SingleCellExperiment` etc.) but unfortunately, with AnnData objects, any processing must be done directly on the appropriate data structure (Counts matrix or the observations table).

In [5]:
intestine_idx <- which((adata$obs)$Field1 == "Your subset measure 1" & (adata$obs)$Field2 == "Your subset measure 2")
# Extract the count matrix from the AnnData object
count_matrix <- adata$X
# Extract the observations
obs <- adata$obs

### Subset the data

In [8]:
counts_subset <- count_matrix[intestine_idx, ]
obs_subset <- obs[intestine_idx, ]

## Identify the markers

This is done using the `scoreMarkers` function, which by default returns AUC values for each marker gene per cluster, which will be used by SPOTlight later.

In [10]:
markers <- scoreMarkers(t(counts_subset), obs_subset$category)

Next, the markers are all put into a dataframe. At this stage, optionally, one can filter for only genes that have an AUC value above some threshold. Here it is commented out as later, we use the top 50 markers for each cell type.

In [11]:
mgs_fil <- lapply(names(markers), function(i) {
    x <- markers[[i]]
    # Filter and keep relevant marker genes, those with AUC > 0.8
    # x <- x[x$mean.AUC > 0.8, ]
    # Sort the genes from highest to lowest weight
    x <- x[order(x$mean.AUC, decreasing = TRUE), ]
    # Add gene and cluster id to the dataframe
    x$gene <- rownames(x)
    x$cluster <- i
    data.frame(x)
})
mgs_df <- do.call(rbind, mgs_fil)

## Identify the Highly Variable Genes (HVGs)

The number of HVGs chosen does not greatly affect model performance, but the authors recommend 3000.

In [12]:
dec <- modelGeneVar(t(counts_subset))
hvg <- getTopHVGs(dec, n = 2000)

## Downsample the cells

Here, up to 100 cells are sampled from each cell type. This reduces the computational load at the deconvolution step.
Here, we also extract a vector (`groups_to_use`) that contains the cell type for each cell in the downsampled dataset.

In [14]:
idx <- split(seq(nrow(counts_subset)), obs_subset$category)

n_cells <- 100
cs_keep <- lapply(idx, function(i) {
    n <- length(i)
    if (n < n_cells)
        n_cells <- n
    sample(i, n_cells)
})

new_counts <- counts_subset[unname(unlist(cs_keep)), ]
new_obs <- obs_subset[unname(unlist(cs_keep)), ]
groups_to_use <- new_obs$category

## Convert the processed count matrix

The count matrix, when extracted from AnnData object, is by default a `dgRMatrix` object in R. However, SPOTlight requires a `dgCMatrix` as an input, so it is converted here. The matrix is also transposed, since SPOTlight expects a count matrix that resembles one found in a `SingleCellExperiment` object, where the rows are genes and columns are individual cells.

In [16]:
counts_final <- as(t(new_counts), "CsparseMatrix")

## Run Deconvolution

Here, `weight_id` refers to which column of the marker genes dataframe contains the weights of the marker genes (in this case, `mean.AUC`). `group_id` refers to which column contains the cell type, and `gene_id` to which column contains the gene names.

`n_top` refers to whether we would like to choose the top `n` marker genes from each cell type (as opposed to selecting those marker genes that pass a threshold). By default, it is `NULL` and so uses the entire marker genes dataframe, but if `n_top` is passed to the function as a value, it will select them from the dataframe.

In [ ]:
res <- SPOTlight(
    x = counts_final,
    y = spe,
    groups = groups_to_use,
    mgs = mgs_df,
    n_top = 50,
    scale = TRUE,
    hvg = hvg,
    weight_id = "mean.AUC",
    group_id = "cluster",
    gene_id = "gene",
    verbose = TRUE)

## Extract the final deconvolution results

`mat` is a matrix that contains predicted proportions of cell types in each spot on the spatial dataset.
`mod` contains details of the final NMF model that was trained.

In [19]:
mat <- res$mat
mod <- res$NMF

## Examine Topic Mapping

With the below plot, the mapping of topics to unique cell types can be examined. Some cell types may be mapped more poorly than others.

In [21]:
plotTopicProfiles(
    x = mod,
    y = groups_to_use,
    facet = FALSE,
    min_prop = 0.01,
    ncol = 1) +
    theme(aspect.ratio = 1)

## Correlation matrix between cell type proportions in spots

In [23]:
plotCorrelationMatrix(mat)

## Plot interactions between cell types as a network graph

In [ ]:
network_interaction <- plotInteractions(mat, "network")

## Prepare a colour palette to plot the spatial scatterpie

In [ ]:
ct <- colnames(mat)

mat[mat < 0.1] <- 0

pal <- colorRampPalette(paletteMartin)(length(ct))
names(pal) <- ct
#swatch(pal) (if you want to check the palette first)

## Plot the spatial scatterpie

In [29]:
plotSpatialScatterpie(
    x = spe,
    y = mat,
    cell_types = colnames(mat),
    img = TRUE,
    scatterpie_alpha = 1,
    pie_scale = 0.4,
    axis = "h",
    degrees = 270) +
    scale_fill_manual(
        values = pal, 
        breaks = names(pal)
        )

## Plot the residual sums of squares (RSS) for each spot

In [31]:
spe$res_ss <- res[[2]][colnames(spe)]
xy <- spatialCoords(spe)
spe$x <- xy[, 1]
spe$y <- xy[, 2]
ggcells(spe, aes(x, y, color = res_ss)) +
    geom_point() +
    scale_color_viridis_c() +
    coord_fixed() +
    theme_bw()

## Visualize the interactions between cell types in two types of interaction plots

In [43]:
interact_heatmap <- plotInteractions(mat, which = "heatmap", metric = "prop")
interact_jaccard <- plotInteractions(mat, which = "heatmap", metric = "jaccard")

## Plot the proportions of one cell type across the spots

This is a useful way to visualize cell type proportions of individual cell types at a time. This is useful to look at patterns of expression of some specific cell types across the tissue.
It can also be useful to visualize them in this way to compare them with the results of other deconvolution softwares, which typically plot their results similarly.

In [72]:
spe$celltype1 <- mat[, which(colnames(mat) == "Cell type 1")]
spe$celltype2 <- mat[, which(colnames(mat) == "Cell type 2")]

ggcells(spe, aes(y, x, color = celltype1)) +
    geom_point(alpha = 1) +
    scale_color_viridis_c(option = "magma", begin = 0, end = 1) +
    coord_fixed() +
    theme_bw() +
    theme(panel.grid.major = element_blank())

ggcells(spe, aes(y, x, color = celltype2)) +
    geom_point(alpha = 1) +
    scale_color_viridis_c(option = "magma", begin = 0, end = 1) +
    coord_fixed() +
    theme_bw() +
    theme(panel.grid.major = element_blank())

In [2]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /media/gambino/students_workdir/ibp/gautam/miniconda3/envs/r_kernel/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_BE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_BE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_BE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_BE.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Brussels
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] Polychrome_1.5.1            ggcorrplot_0.1.4.1         
 [3] Matrix_1.6-3                dplyr_1.1.4                
 [5] Seurat_5.0.1                SeuratObject_5.0.1   